In [ ]:
import sys
import xarray as xr
import pandas as pd
from pathlib import Path
from pyswmm import Simulation, Nodes, Output, SimulationPreConfig
from pyswmm import Simulation, Nodes, Links, Subcatchments, SystemStats, Simulation
import matplotlib.pyplot as plt

sys.path.append(r'C:\Users\everett\Documents\GitHub\camus_to')


data_file = r"C:\Users\everett\Documents\GitHub\camus_to\data\clean\camus_to.nc"

from swmmio import Model

In [ ]:
import numpy as np

class KalmanFilter:
    def __init__(self, state_dim, obs_dim):
        # Initialize dimensions
        self.state_dim = state_dim
        self.obs_dim = obs_dim

        # Initialize state estimate and covariance
        self.x = np.zeros((state_dim, 1))  # State vector
        self.P = np.eye(state_dim)        # State covariance matrix

        # Initialize process and observation noise
        self.Q = np.eye(state_dim)        # Process noise covariance
        self.R = np.eye(obs_dim)          # Observation noise covariance

        # Initialize state transition and observation matrices
        self.F = np.eye(state_dim)        # State transition matrix
        self.H = np.zeros((obs_dim, state_dim))  # Observation matrix

    def predict(self):
        # Predict the next state
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q

    def update(self, z):
        # Update the state with observation z
        y = z - self.H @ self.x  # Innovation
        S = self.H @ self.P @ self.H.T + self.R  # Innovation covariance
        K = self.P @ self.H.T @ np.linalg.inv(S)  # Kalman gain

        self.x = self.x + K @ y  # Update state estimate
        self.P = (np.eye(self.state_dim) - K @ self.H) @ self.P  # Update covariance

# Example usage
kf = KalmanFilter(state_dim=4, obs_dim=2)
kf.F = np.array([[1, 1, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 1, 1],
                 [0, 0, 0, 1]])  # Example state transition matrix
kf.H = np.array([[1, 0, 0, 0],
                 [0, 0, 1, 0]])  # Example observation matrix

# Simulate a prediction and update step
kf.predict()
observation = np.array([[5], [3]])  # Example observation
kf.update(observation)

print("Updated state estimate:")
print(kf.x)

In [ ]:

from pyswmm import Node
from pyswmm import Simulation, Nodes, Links, Subcatchments, SystemStats
from pyswmm import toolkitapi as tka

model_file = Path(rf"C:\Users\everett\Documents\GitHub\camus_to\data\models\swmm\HY013\HY013.inp")
discharge = pd.read_pickle(rf"C:\Users\everett\Documents\GitHub\camus_to\data\models\swmm\HY013\targets.pkl")["discharge(cms)"]
discharge.index = pd.to_datetime(discharge.index)




Qobs = []
Qs = []
Qs_corrected = []
with Simulation(str(model_file)) as sim:
    for ii, step in enumerate(sim):
        
        # Define current_time as an example timestamp or use the appropriate value
        current_time = discharge.index[0]  # Replace with the desired timestamp if needed

        closest_index = discharge.index.get_indexer([pd.Timestamp(current_time)], method='nearest')[0]
        closest_discharge = discharge.iloc[closest_index]
        Qobs.append(closest_discharge)
        
        Qs.append(sim._model.getNodeResult("HY013", tka.NodeResults.newLatFlow.value))
        current_time = sim.current_time
        if ii > 10000:
            kf = KalmanFilter(state_dim=1, obs_dim=1)
            kf.F = np.array([[1]])  # State transition matrix
            kf.H = np.array([[1]])  # Observation matrix
            kf.Q = np.array([[0.1]])  # Process noise covariance
            kf.R = np.array([[0.1]])  # Observation noise covariance

            kf.x = np.array([[Qs[0]]])  # Initialize state with the last inflow value
            kf.P = np.array([[1]])  # Initialize state covariance

            observation = np.array([Qs[ii-10000]])  # Use the inflow from 5 steps ago as the observation
            kf.update(observation)  # Update the Kalman filter with the observation

            corrected_Q = kf.x[0, 0]  # Extract the corrected inflow
            #sim._model.setNodeInflow("HY013", corrected_Q)

            Qs_corrected.append(corrected_Q)
        else:
            Qs_corrected.append(Qs[ii])

